# Authors: Andrea ZANON, Hamza ZERHOUNI

# Problems

consider how big groups must be
could assume that we give teams a 1 week rest period... but that's not realistic

In [1]:
using DataFrames
using JuMP, Gurobi
using DataFrames

# Single league that scales

In [2]:
# number of teams
N = 20

# number of weeks
W = N-1
# number of days per weeks
D = 7; # we suppose week starts on tuesday, beacause monday is actually part of the previous football week


We define the matrix $C \in R^{TxT}$, with $C_{ij} = \frac{1}{j-i}$

In [22]:
function GetC()
    C = zeros((D*(W+1), D*(W+1)))

    for t in 1:W*(D)
        for t_ in t+1:W*(D)
            C[t,t_] = 1/sqrt(t_ - t)
        end
    end

    return C
end

GetC (generic function with 1 method)

Simple model, we are considering single league and making assumption that each team pla

In [53]:
function calendar_week(x,week)
    df = DataFrame(Player=1:N, Monday=0.0, Tuesday=0.0, Wednesday=0.0, Thursday=0.0, Friday=0.0, Saturday=0.0, Sunday=0.0)
    for i=1:N
        for d=1:D
            for j=1:N
                if x[i,j,week,d] == 1.0
                    if d+2 == 9   
                        df[i, 2] = j
                    else
                        df[i,d+2] = j
                    end
                end
            end
        end
    end
    return df
end

calendar_week (generic function with 1 method)

In [87]:
# pass to functions: which teams i are within that group
function GamesBetweenGroups(i_in, i_end, j_in, j_end, w_in, w_end, D)

    # define model
    model = Model(Gurobi.Optimizer)

    # get C
    C = GetC()

    # take slicing of matrix C corresponding to the initial day and end day
    # c_in = 7 * (w_in - 1) + 1 # initial day (d = 1) of week w_in
    # c_end = 7 * (w_end - 1) + D # end day (d = D) of week w_end
    # C = C[c_in:c_end, c_in:c_end]

    # define variables
    # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, x[i = i_in:i_end, j = j_in:j_end, w = w_in:w_end, d = 1:D], Bin)

    # define objective
    # remember week begins on tuesday, this is why w_ goes from w+1
    @objective(model, Min, sum(C[7 * (w-1) + d, 7 * w + d_]* x[i, j, w, d] * x[i, k, w+1, d_] for i in i_in:i_end, 
            j = j_in:j_end, k = j_in:j_end, w = w_in:(w_end-1), d = 1:D, d_ = 1:D))

    # each team from one group must play exactly once against each team in the other group
    @constraint(model, [i = i_in:i_end, j = j_in:j_end], sum(x[i, j, w, d] for d = 1:D,w = w_in:w_end) == 1)

    # no two teams i can play against the same team j
    @constraint(model, [j = j_in:j_end, d = 1:D, w = w_in:w_end], sum(x[i, j, w, d] for i = i_in:i_end) <= 1)

    # can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
    @constraint(model, [i = i_in:i_end, j = j_in:j_end, w = w_in:w_end, d = 1:3], x[i, j, w, d] == 0)

    # solve model
    optimize!(model)
    
    return value.(x)

end

GamesBetweenGroups (generic function with 2 methods)

In [106]:
function GamesWithinGroups(N_in, N_end, w_in, w_end, D)
    
    # create model
    model = Model(Gurobi.Optimizer)

    # get C
    C = GetC()

    print(1)

    # define variables
    # 1 if team i plays team j on day d of week w, 0 otherwise
    @variable(model, x[i = N_in:N_end, j = N_in:N_end, w = w_in:w_end, d = 1:D], Bin)

    print(2)

    # define objective
    # remember week begins on tuesday, this is why w_ goes from w+1
    @objective(model, Min, sum(C[7 * (w-1) + d, 7 * (w) + d_]* x[i, j, w, d] * x[i, k, w+1, d_] for i = N_in:N_end, 
        j = N_in:N_end, k = N_in:N_end, w = w_in:(w_end-1), d = 1:D, d_ = 1:D))


    print(3)
    # define constraints
    # each team can never play itself
    @constraint(model, [i = N_in:N_end, w = w_in:w_end, d = 1:D], x[i, i, w, d] == 0)

    print(4)
    # if team A plays team B, then team B plays team A
    @constraint(model, [i = N_in:N_end, j = N_in:N_end, w = w_in:w_end, d = 1:D], x[i, j, w, d] == x[j, i, w, d])

    print(5)
    # can't play on tuesday (day 1), wednesday (day 2), thursday (day 3)
    @constraint(model, [i = N_in:N_end, j = N_in:N_end, w = w_in:w_end, d = 1:3], x[i, j, w, d] == 0)

    print(6)

    # all teams need to play each other exactly once
    for i = N_in:N_end
        for j = N_in:N_end
            if i != j
                @constraint(model, sum(x[i, j, w, d] for d = 1:D, w = w_in:w_end) == 1)
            end
        end
    end

    print(7)
    # no two teams i can play against the same team j
    @constraint(model, [j = N_in:N_end, d = 1:D, w = w_in:w_end], sum(x[i, j, w, d] for i = N_in:N_end) <= 1)

    print(8)
    # solve model
    optimize!(model)

    return value.(x)
end

GamesWithinGroups (generic function with 1 method)

In [107]:
X = GamesWithinGroups(1, 5, 16, 19, 7)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
12345678Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1300 rows, 700 columns and 2820 nonzeros
Model fingerprint: 0x2cf1d53d
Model has 18375 quadratic objective terms
Variable types: 0 continuous, 700 integer (700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [6e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.6176198
Presolve removed 1210 rows and 540 columns
Presolve time: 0.01s
Presolved: 170 rows, 240 columns, 4000 nonzeros
Variable types: 80 continuous, 160 integer (160 binary)

Root relaxation: objective 0.000000e+00, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl

4-dimensional DenseAxisArray{Float64,4,...} with index sets:
    Dimension 1, 1:5
    Dimension 2, 1:5
    Dimension 3, 16:19
    Dimension 4, Base.OneTo(7)
And data, a 5×5×4×7 Array{Float64, 4}:
[:, :, 16, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 17, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 18, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 19, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 16, 2] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 17, 2] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.

In [108]:
for i = 1:5
    for j = 1:5
        for w = 16:19
            for d = 1:7
                if X[i, j, w, d] == 1.0
                    println("Team ", i, " plays team ", j, " on day ", d, " of week ", w)
                end
            end
        end
    end
end

Team 1 plays team 2 on day 5 of week 17
Team 1 plays team 3 on day 4 of week 17
Team 1 plays team 4 on day 7 of week 19
Team 1 plays team 5 on day 5 of week 19
Team 2 plays team 1 on day 5 of week 17
Team 2 plays team 3 on day 6 of week 17
Team 2 plays team 4 on day 5 of week 19
Team 2 plays team 5 on day 7 of week 19
Team 3 plays team 1 on day 4 of week 17
Team 3 plays team 2 on day 6 of week 17
Team 3 plays team 4 on day 4 of week 19
Team 3 plays team 5 on day 7 of week 17
Team 4 plays team 1 on day 7 of week 19
Team 4 plays team 2 on day 5 of week 19
Team 4 plays team 3 on day 4 of week 19
Team 4 plays team 5 on day 4 of week 17
Team 5 plays team 1 on day 5 of week 19
Team 5 plays team 2 on day 7 of week 19
Team 5 plays team 3 on day 7 of week 17
Team 5 plays team 4 on day 4 of week 17
